<h1 align="center">Using Matricies as Linear Transformations to Perform Rotations of Images</h1>
<h2 align="center">Remi L.</h2>

Matrices are powerful tools for representing transformations in mathematics, particularly in geometry. They can be thought of as functions that modify vectors in space, where the vector represents a point or direction. In this case, we will be working with 2D points that represent pixels in an image, though the concept extends to higher dimensions as well. A matrix applied to a vector can perform a variety of operations, such as translating, scaling, or rotating the vector. In linear algebra, these matrices that act as functions are called *transformations*.

Some common transformations that can be expressed using matrices include **shear matrices** (which tilt the vector in one direction), **scaling matrices** (which resize the vector by a constant factor), and **rotation matrices** (which rotate the vector around the origin by a specified angle). Each type of transformation affects the components of a vector in predictable ways.

In the case of rotation, we want to rotate an $(x, y)$ point around an angle $\theta$. We may define the transformation as follows:

$$
T(\vec{x}) = A\vec{x} =
\begin{bmatrix}
\cos \theta & -\sin \theta \\
\sin \theta & \cos \theta
\end{bmatrix}\vec{x}$$
where $\vec{x}$ is a vector in $\mathbb{R}^2$.

# Why?

Recall that a transformation vector is defined in terms of where the standard basis vectors in that vector space get sent. For a matrix that serves as a transformation for vectors in $\mathbb{R}^2$, the transformation matrix can be defined generically as: $$
\begin{bmatrix}
| & | \\
T(e_1) & T(e_2) \\
| & |
\end{bmatrix}
$$

In this particular case, we are interested in what happens to a vector that gets sent through a transformation that rotates it about an angle $\theta$. This may be better explained by an example.

Consider the standard basis vectors in $\mathbb{R}^2$:

$$A = \begin{bmatrix}
1 & 0 \\
0 & 1
\end{bmatrix}
$$

If we are interested in rotating these vectors about the angle $\theta = \frac{\pi}{2}$ (i.e. rotating the standard basis vectors 90°, or $\frac{\pi}{2}$ rad), these vectors now become

$$A^\prime = \begin{bmatrix}
0 & -1 \\
1 & 0
\end{bmatrix}
$$

The standard basis are sent to $(-1, 0)$ and $(0, 1)$. This defines our transformation since we know exactly where the standard basis vectors are being sent. Looking back at our generic formula from earlier, we set $\theta = \frac{\pi}{2}$:

$$
A =
\begin{bmatrix}
\cos \theta & -\sin \theta \\
\sin \theta & \cos \theta \\
\end{bmatrix} = 
\begin{bmatrix}
\cos(\frac{\pi}{2}) & -\sin(\frac{\pi}{2}) \\
\sin(\frac{\pi}{2}) & \cos(\frac{\pi}{2}) \\
\end{bmatrix} = 
\begin{bmatrix}
0 & -1 \\
1 & 0 \\
\end{bmatrix} = A^\prime
$$

When this transformation matrix is applied to any vector or point in $\mathbb{R}^2$, it will effectively rotate that vector 90°.

With the specifics aside, we can now work towards implementing a rotation matrix using Python and some third-party libraries. First we start with installing Pillow which will allow us to access the pixel values of our image:

In [1]:
pip install pillow


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


Next, we can import pillow into our project for use. Note that in this case, we import from the PIL package (Python Imaging Library) which comes from installing pillow. 

We will also need to make use of Python's built-in `math` library, in order to use these trigonometric functions.

While we're at it, we'll import the `Image` class. The `Image` class lets us perform specific operations on images (e.g getting pixel values, dimensions, etc.)

In [2]:
from PIL import Image # Import the Image class from the pillow PIL library
import math # Import the math library

Next, let's define a location for our image file: 

In [29]:
image_path = "./bocchi.jpg" # Looks for a file named my_image.png in the current directory

Here we define a few operations for accessing our data:

In [40]:
image = Image.open(image_path) # Open our image into a pillow image object.
image_data = image.load() # This returns data containing our pixels and other information about the image we just loaded.
width, height = image.size
print(f"Width = {width}, Height = {height}")

Width = 736, Height = 717


This is where it starts to get complicated. It should be noted that for context, we are going to treat each pixel in the image as an $(x, y)$ coordinate pair corresponding to $(row, col)$. Think of the $A_{ij}$ notation for matrix entries. In the realm of computer graphics, this is referred to as _point based rendering_.

Here we  define a function that takes in our $(x, y)$ point and spits out its new transformed coordinate. This is essentially a function implementation of our rotation matrix multiplied by the $(x, y)$ vector.

A couple of things to note: 
1. Since these trig functions return a decimal number, we simply round them to the nearest whole number by calling `round()`.

2. We also negate the new coordinate. This is because it doesn't make sense to have a new pixel location of $(-1, -1)$. By negating each coordinate, we are translating it back into the first quadrant (all positive coordinates, which correspond to positive matrix/array indices).

In [49]:
def rotate_90(x, y):
    # Without the negatives on both of these, these indices will be negative. We want only positive indices.
    theta = math.pi/2 # 90º
    x_new = -round(x*math.cos(theta) - y*math.sin(theta))
    y_new = -round(x*math.sin(theta) + y*math.cos(theta))
    return (x_new, y_new)

Next, we're going to define a blank canvas. The thought process here is that we're going to pluck pixels from our image and plot them on this blank canvas. This is necessary if you're dealing $m$ $*$ $n$ non-square images.

Here, we say that the width of our blank canvas is going to be the height of our original image, and the height of our blank canvas is going to be the width of our original image. Can you see why? (Hint: Take a rectangular piece of paper and rotate it 90º.)

In [ ]:
# Creates a blank canvas with dimensions height x width. This is how the dimensions are once the image is rotated 90º.
new_image = Image.new('RGB', (height, width))

We're also going to define two loops. The outer loop will loop in the x direction. The inner loop will iterate in the `y` direction for every value of `x`. This will essentially give us every $(x, y)$ coordinate for every pixel in our image.

For every $(x, y)$ pixel coordinate we come across, we are going to perform the rotation of that coordinate by calling the `rotate_90(x, y)` function which will give us the new coordinates of each pixel: $(x_{new}, y_{new})$.

`pixel = image.getpixel((x,y))`: This gets the pixel from our image at the point $(x, y)$.

`new_image.putpixel((x_new, y_new), pixel)`: This takes that pixel and sticks it in its new position.

In [50]:
for x in range(width):
    for y in range(height):
            # Determine new coordinates where pixel will be sent.
            # This only works if the angle is 90º. If the angle is 180º, then new_image should be initialized with (width, height) instead.
            x_new, y_new = rotate_90(x,y) 
            # Get pixel at (x, y)
            pixel = image.getpixel((x,y))
            # Put that pixel in the new position
            new_image.putpixel((x_new, y_new), pixel)

Let's see our rotated image!

In [51]:
new_image.show()